In [1]:
pip install imbalanced-learn

     |████████████████████████████████| 199 kB 16.3 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 44.3 MB/s eta 0:00:01
     |████████████████████████████████| 30.8 MB 25.1 MB/s eta 0:00:01| 17.6 MB 25.1 MB/s eta 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 0.17.0
    Uninstalling joblib-0.17.0:
      Successfully uninstalled joblib-0.17.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lale 0.6.10 requires scikit-learn<=1.0.2, but you have scikit-learn 1.1.3 which is incompatible.
autoai-ts-libs 1.1.9 requires scikit-learn<1.1,>=1.0.2; python_version >= "3.9", but you have scikit-learn 1.1.3 which is incompatible.
autoai-libs 1.13.6 req

In [2]:
import numpy as np 
import pandas as pd
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import  GradientBoostingClassifier


import warnings
warnings.filterwarnings('ignore')

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='RKxS-0l3JX7jZnwU1RUeCMPrpqpr_B_G4zMw5GPfvyKZ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'custommodeldeployment-donotdelete-pr-vyf8ll0x1vinyz'
object_key = 'loan_prediction.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [5]:
numerical_features = df.select_dtypes(include = [np.number]).columns
categorical_features = df.select_dtypes(include = [np.object]).columns

In [6]:
df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0])
df['Married'] = df['Married'].fillna(df['Married'].mode()[0])
df['Dependents'] = df['Dependents'].str.replace('+','')
df['Dependents'] = df['Dependents'].fillna(df['Dependents'].mode()[0])
df['LoanAmount'] = df['LoanAmount'].fillna(df['LoanAmount'].mode()[0])
df['Self_Employed'] = df['Self_Employed'].fillna(df['Self_Employed'].mode()[0])
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0])
df['Credit_History'] = df['Credit_History'].fillna(df['Credit_History'].mode()[0])

In [7]:
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [8]:
df['Gender'].replace({'Male':1,'Female':0},inplace=True)
df['Dependents'].replace({'0':0,'1':1,'2':2,'3':3},inplace=True)
df['Married'].replace({'Yes':1,'No':0},inplace=True)
df['Self_Employed'].replace({'Yes':1,'No':0},inplace=True)
df['Property_Area'].replace({'Urban':2,'Rural':0,'Semiurban':1},inplace=True)
df['Education'].replace({'Graduate':1,'Not Graduate':0},inplace=True)
df['Loan_Status'].replace({'Y':1,'N':0},inplace=True)

In [9]:
df['CoapplicantIncome']=df['CoapplicantIncome'].astype("int64")
df['LoanAmount']=df['LoanAmount'].astype("int64")
df['Loan_Amount_Term']=df['Loan_Amount_Term'].astype("int64")
df['Credit_History']=df['Credit_History'].astype("int64")

In [10]:
le = LabelEncoder()
df['Loan_ID'] = le.fit_transform(df.Loan_ID)
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,1,0,0,1,0,5849,0,120,360,1,2,1
1,1,1,1,1,1,0,4583,1508,128,360,1,0,0
2,2,1,1,0,1,1,3000,0,66,360,1,2,1
3,3,1,1,0,0,0,2583,2358,120,360,1,2,1
4,4,1,0,0,1,0,6000,0,141,360,1,2,1


In [11]:
smote = SMOTETomek(0.90)

y = df['Loan_Status']
x = df.drop(columns=['Loan_Status'],axis=1)
x_bal,y_bal = smote.fit_resample(x,y)
print(y.value_counts())
print(y_bal.value_counts())

1    422
0    192
Name: Loan_Status, dtype: int64
1    352
0    309
Name: Loan_Status, dtype: int64


In [12]:
scaler = StandardScaler()
x_bal = scaler.fit_transform(x_bal)
x_bal = pd.DataFrame(x_bal)
x_bal.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-1.748105,0.539459,-1.170118,-0.720265,0.616408,-0.338631,0.125871,-0.525115,-0.304091,0.297788,0.579097,1.319475
1,-1.742215,0.539459,0.854615,0.321519,0.616408,-0.338631,-0.106418,-0.038541,-0.200554,0.297788,0.579097,-1.268576
2,-1.736325,0.539459,0.854615,-0.720265,0.616408,2.953064,-0.396871,-0.525115,-1.002965,0.297788,0.579097,1.319475
3,-1.730434,0.539459,0.854615,-0.720265,-1.622303,-0.338631,-0.473383,0.235722,-0.304091,0.297788,0.579097,1.319475
4,-1.724544,0.539459,-1.170118,-0.720265,0.616408,-0.338631,0.153577,-0.525115,-0.032306,0.297788,0.579097,1.319475


In [13]:
x_train, x_test, y_train, y_test = train_test_split(x_bal, y_bal, test_size = 0.33, random_state = 42)

# Compare the Models - KNN, Decision Tree, Random Forest CLassifier, XGBoost

## KNN

In [14]:
def kneighborsClassifier(x_train, x_test, y_train, y_test):
    knn = KNeighborsClassifier() 
    knn.fit(x_train,y_train)
    yPred = knn.predict(x_test)
    print("****KNN****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=knn.predict(x_test)
    y_pred1=knn.predict(x_train)
    knn_test_accuracy = accuracy_score(y_test,y_pred)
    knn_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ',knn_test_accuracy)
    print('Training accuracy: ', knn_train_accuracy)

kneighborsClassifier(x_train, x_test, y_train, y_test)

****KNN****
Confusion matrix
[[73 31]
 [20 95]]
Classification report
              precision    recall  f1-score   support

           0       0.78      0.70      0.74       104
           1       0.75      0.83      0.79       115

    accuracy                           0.77       219
   macro avg       0.77      0.76      0.76       219
weighted avg       0.77      0.77      0.77       219

Testing accuracy:  0.7671232876712328
Training accuracy:  0.832579185520362


# Decision Tree

In [15]:
def decisionTreeClassifier(x_train, x_test, y_train, y_test):
    dt = DecisionTreeClassifier() 
    dt.fit(x_train,y_train)
    yPred = dt.predict(x_test)
    print("****DecisionTreeClassifier****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=dt.predict(x_test)
    y_pred1=dt.predict(x_train)
    decision_tree_test_accuracy = accuracy_score(y_test,y_pred)
    decision_tree_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', decision_tree_test_accuracy)
    print('Training accuracy: ', decision_tree_train_accuracy)

decisionTreeClassifier(x_train, x_test, y_train, y_test)

****DecisionTreeClassifier****
Confusion matrix
[[82 22]
 [26 89]]
Classification report
              precision    recall  f1-score   support

           0       0.76      0.79      0.77       104
           1       0.80      0.77      0.79       115

    accuracy                           0.78       219
   macro avg       0.78      0.78      0.78       219
weighted avg       0.78      0.78      0.78       219

Testing accuracy:  0.7808219178082192
Training accuracy:  1.0


## Random Forest CLassifier

In [16]:
def randomForestClassifier(x_train, x_test, y_train, y_test):
    rf = RandomForestClassifier() 
    rf.fit(x_train,y_train)
    yPred = rf.predict(x_test)
    print("****RandomForestClassifier****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=rf.predict(x_test)
    y_pred1=rf.predict(x_train)
    random_forest_test_accuracy = accuracy_score(y_test,y_pred)
    random_forest_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', random_forest_test_accuracy)
    print('Training accuracy: ',random_forest_train_accuracy)

randomForestClassifier(x_train, x_test, y_train, y_test)

****RandomForestClassifier****
Confusion matrix
[[ 82  22]
 [ 13 102]]
Classification report
              precision    recall  f1-score   support

           0       0.86      0.79      0.82       104
           1       0.82      0.89      0.85       115

    accuracy                           0.84       219
   macro avg       0.84      0.84      0.84       219
weighted avg       0.84      0.84      0.84       219

Testing accuracy:  0.8401826484018264
Training accuracy:  1.0


# XGBoost

In [17]:
def xgboost(x_train, x_test, y_train, y_test):
    xg = GradientBoostingClassifier() 
    xg.fit(x_train,y_train)
    yPred = xg.predict(x_test)
    print("****Gradient BoostingClassifier****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=xg.predict(x_test)
    y_pred1=xg.predict(x_train)
    xgboost_test_accuracy = accuracy_score(y_test,y_pred)
    xgboost_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', xgboost_test_accuracy)
    print('Training accuracy: ', xgboost_train_accuracy)
  
xgboost(x_train, x_test, y_train, y_test)

****Gradient BoostingClassifier****
Confusion matrix
[[ 73  31]
 [ 11 104]]
Classification report
              precision    recall  f1-score   support

           0       0.87      0.70      0.78       104
           1       0.77      0.90      0.83       115

    accuracy                           0.81       219
   macro avg       0.82      0.80      0.80       219
weighted avg       0.82      0.81      0.81       219

Testing accuracy:  0.8082191780821918
Training accuracy:  0.9457013574660633


In [18]:
kneighborsClassifier(x_train, x_test, y_train, y_test)
decisionTreeClassifier(x_train, x_test, y_train, y_test)
randomForestClassifier(x_train, x_test, y_train, y_test)
xgboost(x_train, x_test, y_train, y_test)

****KNN****
Confusion matrix
[[73 31]
 [20 95]]
Classification report
              precision    recall  f1-score   support

           0       0.78      0.70      0.74       104
           1       0.75      0.83      0.79       115

    accuracy                           0.77       219
   macro avg       0.77      0.76      0.76       219
weighted avg       0.77      0.77      0.77       219

Testing accuracy:  0.7671232876712328
Training accuracy:  0.832579185520362
****DecisionTreeClassifier****
Confusion matrix
[[81 23]
 [23 92]]
Classification report
              precision    recall  f1-score   support

           0       0.78      0.78      0.78       104
           1       0.80      0.80      0.80       115

    accuracy                           0.79       219
   macro avg       0.79      0.79      0.79       219
weighted avg       0.79      0.79      0.79       219

Testing accuracy:  0.7899543378995434
Training accuracy:  1.0
****RandomForestClassifier****
Confusion matrix
[

**KNN**
1.   Testing accuracy:  0.7955555555555556
2.   Training accuracy:  0.8392070484581498

**DecisionTreeClassifier**
1. Testing accuracy:  0.7822222222222223
2. Training accuracy:  1.0

**RandomForestClassifier**
1. Testing accuracy:  0.8133333333333334
2. Training accuracy:  1.0

**Gradient BoostingClassifier**
1. Testing accuracy:  0.8088888888888889
2. Training accuracy:  0.9405286343612335

RandomForestClassifier is chosen for further process.

# Evaluating Performance Of The Model And Saving The Model

In [19]:
from sklearn.model_selection import cross_val_score
forest_reg = RandomForestClassifier(n_estimators = 10, criterion = "entropy", random_state = 42) 
forest_reg.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [20]:
yPred = forest_reg.predict(x_test)
y_pred=forest_reg.predict(x_test)
y_pred1=forest_reg.predict(x_train)
random_forest_test_accuracy = accuracy_score(y_test,y_pred)
random_forest_train_accuracy = accuracy_score(y_train,y_pred1)
print('Testing accuracy: ', random_forest_test_accuracy)
print('Training accuracy: ',random_forest_train_accuracy)
f1_score(yPred,y_test, average='weighted')
cv = cross_val_score(forest_reg,x,y,cv=5)
np.mean(cv)

Testing accuracy:  0.817351598173516
Training accuracy:  0.9909502262443439


0.723057443689191

In [21]:
!pip install ibm_watson_machine_learning
!pip install watson_machine_learning_client

     |████████████████████████████████| 538 kB 15.9 MB/s eta 0:00:01


In [22]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url" : "https://us-south.ml.cloud.ibm.com",
    "apikey" : "dfw_FbocYtstRSnPfHvfIkPS_-QeBamFcyD3X7wHK-cH"
}

client=APIClient(wml_credentials)

In [23]:
def guid_from_space_name(client, space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [24]:
space_uid=guid_from_space_name(client,'models')
print("Space UID = "+space_uid)

Space UID = 782c84e1-22d2-41f9-acf4-8ed9f9fc170a


In [25]:
client.set.default_space(space_uid)

'SUCCESS'

In [26]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [33]:
software_spec_uid = client.software_specifications.get_uid_by_name("default_py3.7")
software_spec_uid

'e4429883-c883-42b6-87a8-f419d64088cd'

In [35]:
model_details = client.repository.store_model(
    model=forest_reg,meta_props={
    client.repository.ModelMetaNames.NAME: "Loan_Prediction",
    client.repository.ModelMetaNames.TYPE:"scikit-learn_0.20",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid }
                                           )
model_id = client.repository.get_model_uid(model_details)

Traceback (most recent call last):
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/libs/repo/mlrepositoryclient/model_collection.py", line 254, in _save_scikit_pipeline_model
    model_artifact = self._create_pipeline_model(artifact, query_param)
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/libs/repo/mlrepositoryclient/model_collection.py", line 514, in _create_pipeline_model
    model_artifact = self._create_pipeline_model_v4_cloud(model_artifact, query_param)
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/libs/repo/mlrepositoryclient/model_collection.py", line 1005, in _create_pipeline_model_v4_cloud
    model_output = self.repository_api.ml_assets_model_creation_v4_cloud(model_input, query_param,headers)
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/libs/repo/swagger_client/apis/repository_api.py", line 301,

WMLClientError: Publishing model failed.
Reason: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 13 Nov 2022 15:53:58 GMT', 'Content-Type': 'application/json', 'Content-Length': '292', 'Connection': 'keep-alive', 'server-timing': 'intid;desc=2951cdd70b527ad7, intid;desc=2951cdd70b527ad7', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'CF-Cache-Status': 'DYNAMIC', 'Server': 'cloudflare', 'CF-RAY': '7698aee8cb50e75e-DFW'})
HTTP response body: {
  "trace": "7c5cabc5fe8e83cfc13fa5f1ec31ad87",
  "errors": [{
    "code": "invalid_request_entity",
    "message": "Invalid request entity: Unsupported software specification 'default_py3.7'.",
    "more_info": "https://cloud.ibm.com/apidocs/machine-learning"
  }],
  "status_code": "400"
}


In [ ]:
model_id